<a href="https://colab.research.google.com/github/cmarsay/exoplanetColab/blob/main/exoplanet_stylegan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2 - Fine tuning and interpolation

This following code has been adapted from the colab at this [link](https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PyTorch.ipynb).

## Setup

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-369dee2a-d6c5-5230-e783-93c276e1d554)


Let’s connect our Google Drive account.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
     |████████████████████████████████| 108 kB 5.7 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 12.8 MB 50.7 MB/s 
     |████████████████████████████████| 15.7 MB 50.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 

You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Already up to date.
Saved working directory and index state WIP on main: 464100c lil cleanup


## Dataset Preparation

Upload a .zip of square images to Google Drive.  Mine is in a folder called `jelly_square.zip'. First make a folder to contain our new dataset in the datasets folder.  I've called mine 'square_images'.

Make new folder in colab-sg2-ada-pytorch folder





In [ ]:
os.mkdir('/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets')

NameError: ignored

Next run the following code to convert your images to the correct format and move them to the `datasets` folder.

In [ ]:
!python /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/AI/DATASETS/terradataset.zip --dest /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/terraset

100% 300/300 [00:05<00:00, 58.52it/s]


In [ ]:
# # move square images from Google Drive to the datasets folder of styleGAN2 project
# !mv '/content/drive/MyDrive/AI/DATASETS/terradataset.zip' '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets'

## Add Pre-trained Model

Download a pre-trained model from [here](https://github.com/justinpinkney/awesome-pretrained-stylegan2#panda). Upload it to your Google Drive.  Then move it to the `pretrained` folder with the following code (NB you'll need to change the name of your pretrained model).  


In [ ]:
!mv '/content/drive/MyDrive/02network-snapshot-001108.pkl' '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained'

## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/terra/00000'
resume_from = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00000-00000-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000024.pkl'
# resume_from = './pretrained/network-snapshot-027750.pkl'
aug_strength = 0.0
train_count = 1000
mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 3

In [ ]:
!python train.py  --metrics=fid50k_full --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 3,
  "network_snapshot_ticks": 3,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/terra/00000",
    "use_labels": false,
    "max_size": 300,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "chann

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Testing/Inference

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of usinng your trained model to generate new material, usually images or videos.

### Generate Single Images

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
!python generate.py --outdir=/content/drive/MyDrive/A.I --trunc=0.8 --seeds=100 --network='/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/02network-snapshot-001108.pkl'

Loading networks from "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/02network-snapshot-001108.pkl"...
Generating image for seed 100 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


#### Non-Square outputs

We can modify the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.

* `--size` size takes in a value of `xdim-ydim`. For example, to generate a 1920x1080 image use `1920-1080`
* `--scale-type` This determines the padding style to apply in the additional space. There are four options: `pad`, `padside`, `symm`, and `symmside`. I recommend trying each one to see what works best with your images.


In [ ]:
!python generate.py --outdir=/content/drive/MyDrive/AI --trunc=0.7 --size=1920-1080 --scale-type=symm --seeds=0-9 --network='/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/01network-snapshot-000048.pkl'

python3: can't open file 'generate.py': [Errno 2] No such file or directory


We can use these options for any image or video generation commands (excluding projection).

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [ ]:
!python generate.py --process="truncation" --outdir=/content/drive/MyDrive/AI-trav-3/ --start=-0.8 --stop=5 --increment=0.02 --seeds=470 --network='/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/02network-snapshot-001108.pkl'

Loading networks from "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/02network-snapshot-001108.pkl"...
Generating truncation -0.80
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating truncation -0.78
Generating truncation -0.76
Generating truncation -0.74
Generating truncation -0.72
Generating truncation -0.70
Generating truncation -0.68
Generating truncation -0.66
Generating truncation -0.64
Generating truncation -0.62
Generating truncation -0.60
Generating truncation -0.58
Generating truncation -0.56
Generating truncation -0.54
Generating truncation -0.52
Generating truncation -0.50
Generating truncation -0.48
Generating truncation -0.46
Generating truncation -0.44
Generating truncation -0.42
Generating truncation -0.40
Generating truncation -0.38
Generating truncation -0.36
Generating truncation -0.34
Generating truncation -0.32
Generating truncation -0.30
Generating truncation -0.28
Ge

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

### Basic Projector

*   `--target`: this is a path to the image file that you want to "find" in your model. This image must be the exact same size as your model.
*   `--num-steps`: how many iterations the projctor should run for. Lower will mean less steps and less likelihood of a good projection. Higher will take longer but will likely produce better images.



In [ ]:
!python projector.py --help

Usage: projector.py [OPTIONS]

  Project given image to the latent space of
  pretrained network pickle.

  Examples:

  python projector.py --outdir=out --target=~/mytargetimg.png \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Options:
  --network TEXT        Network pickle filename
                        [required]

  --target FILE         Target image file to
                        project to  [required]

  --num-steps INTEGER   Number of optimization
                        steps  [default: 1000]

  --seed INTEGER        Random seed  [default:
                        303]

  --save-video BOOLEAN  Save an mp4 video of
                        optimization progress
                        [default: True]

  --outdir DIR          Where to save the output
                        images  [required]

  --help                Show this message and
                        exit.


In [ ]:
!python projector.py --network=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/01network-snapshot-000048.pkl --outdir=/content/drive/MyDrive/AI --target=/content/drive/MyDrive/AI/Image52.png --num-steps=200 --seed=0

Loading networks from "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/01network-snapshot-000048.pkl"...
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
step    1/200: dist 0.76 loss 9539.21
step    2/200: dist 0.76 loss 13230.89
step    3/200: dist 0.78 loss 11500.81
step    4/200: dist 0.75 loss 8699.06
step    5/200: dist 0.75 loss 5684.88
step    6/200: dist 0.76 loss 3195.88
step    7/200: dist 0.75 loss 1952.39
step    8/200: dist 0.75 loss 2371.83
step    9/200: dist 0.74 loss 3987.54
step   10/200: dist 0.74 loss 5793.94
step   11/200: dist 0.74 loss 6793.99
step   12/200: dist 0.74 loss 6562.34
step   13/200: dist 0.74 loss 5728.27
step   14/200: dist 0.74 loss 4611.97
step   15/200: dist 0.73 loss 3423.93
step   16/200: dist 0.73 loss 2540.16
step   17/200: dist 0.72 loss 2071.25
step   18/200: dist 0.72 loss 1760.93
step   19/200: di

#### Linear Interpolation

In [ ]:
!python generate.py --outdir=/content/out/video1-w-0.5/ --space="z" --trunc=0.5 --process="interpolation" --seeds=463,470 --network='./pretrained/network-snapshot-027750.pkl'

Loading networks from "./pretrained/network-snapshot-027750.pkl"...
Generating image for frame 0/240 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/240 ...
Generating image for frame 2/240 ...
Generating image for frame 3/240 ...
Generating image for frame 4/240 ...
Generating image for frame 5/240 ...
Generating image for frame 6/240 ...
Generating image for frame 7/240 ...
Generating image for frame 8/240 ...
Generating image for frame 9/240 ...
Generating image for frame 10/240 ...
Generating image for frame 11/240 ...
Generating image for frame 12/240 ...
Generating image for frame 13/240 ...
Generating image for frame 14/240 ...
Generating image for frame 15/240 ...
Generating image for frame 16/240 ...
Generating image for frame 17/240 ...
Generating image for frame 18/240 ...
Generating image for frame 19/240 ...
Generating image for frame 20/240 ...
Generating image for frame 21/240 ...

In [ ]:
!python generate.py --outdir=out/video1-w/ --space="w" --trunc=1 --process="interpolation" --seeds=85,265,297,849 --network='./pretrained/network-snapshot-027750.pkl'

Loading networks from "./pretrained/wikiart.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating image for frame 0/720 ...
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/720 ...
Generating image for frame 2/720 ...
Generating image for frame 3/720 ...
Generating image for frame 4/720 ...
Generating image for frame 5/720 ...
Generating image for frame 6/720 ...
Generating image for frame 7/720 ...
Generating image for frame 8/720 ...
Generating image for frame 9/720 ...
Generating image for frame 10/720 ...
Generating image for frame 11/720 ...
Generating image for frame 12/720 ...
Generating image for frame 13/720 ...
Generating image for frame 14/720 ...
Generating image for frame 15/720 ...
Generating image for frame 16/720 ...
Generating image for frame 17/720 ...
Generating image for frame 18/720 ...
Generating image for frame 19/720 ...
Generating image for frame 20/720 ...
Generating image for frame 21/720 ...
Generating imag

#### Slerp Interpolation

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

In [ ]:
!python generate.py --outdir=out/slerp-z/ --space="z" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network='./pretrained/network-snapshot-027750.pkl' --frames=24

Loading networks from "./pretrained/wikiart.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 464, in generate_images
    interpolate(G,device,projected_w,seeds,random_seed,space,truncation_psi,label,frames,noise_mode,dirpath,interpolation,easin

In [ ]:
!python generate.py --outdir=out/slerp-w/ --space="w" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network='./pretrained/network-snapshot-027750.pkl' --frames=12

Loading networks from "./pretrained/wikiart.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating image for frame 0/36 ...
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/36 ...
Generating image for frame 2/36 ...
Generating image for frame 3/36 ...
Generating image for frame 4/36 ...
Generating image for frame 5/36 ...
Generating image for frame 6/36 ...
Generating image for frame 7/36 ...
Generating image for frame 8/36 ...
Generating image for frame 9/36 ...
Generating image for frame 10/36 ...
Generating image for frame 11/36 ...
Generating image for frame 12/36 ...
Generating image for frame 13/36 ...
Generating image for frame 14/36 ...
Generating image for frame 15/36 ...
Generating image for frame 16/36 ...
Generating image for frame 17/36 ...
Generating image for frame 18/36 ...
Generating image for frame 19/36 ...
Generating image for frame 20/36 ...
Generating image for frame 21/36 ...
Generating image for frame 22/36 ...


#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [ ]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network='./pretrained/network-snapshot-027750.pkl'

Loading networks from "./pretrained/network-snapshot-027750.pkl"...
Generating image for frame 0/240 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/240 ...
Generating image for frame 2/240 ...
Generating image for frame 3/240 ...
Generating image for frame 4/240 ...
Generating image for frame 5/240 ...
Generating image for frame 6/240 ...
Generating image for frame 7/240 ...
Generating image for frame 8/240 ...
Generating image for frame 9/240 ...
Generating image for frame 10/240 ...
Generating image for frame 11/240 ...
Generating image for frame 12/240 ...
Generating image for frame 13/240 ...
Generating image for frame 14/240 ...
Generating image for frame 15/240 ...
Generating image for frame 16/240 ...
Generating image for frame 17/240 ...
Generating image for frame 18/240 ...
Generating image for frame 19/240 ...
Generating image for frame 20/240 ...
Generating image for frame 21/240 ...

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
!python generate.py --outdir=/content/drive/MyDrive/A.I --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=90 --network='/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/02network-snapshot-001108.pkl'

Loading networks from "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/02network-snapshot-001108.pkl"...
Generating image for frame 0/721 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/721 ...
Generating image for frame 2/721 ...
Generating image for frame 3/721 ...
Generating image for frame 4/721 ...
Generating image for frame 5/721 ...
Generating image for frame 6/721 ...
Generating image for frame 7/721 ...
Generating image for frame 8/721 ...
Generating image for frame 9/721 ...
Generating image for frame 10/721 ...
Generating image for frame 11/721 ...
Generating image for frame 12/721 ...
Generating image for frame 13/721 ...
Generating image for frame 14/721 ...
Generating image for frame 15/721 ...
Generating image for frame 16/721 ...
Generating image for frame 17/721 ...
Generating image for frame 18/721 ...
Generating image for frame 19/721 ...
Generati